In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import signal

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
video_file = '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4'

In [ ]:
!ffmpeg -hide_banner -i /kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4

In [ ]:
capture = cv.VideoCapture(video_file)
print(f'FPS: {capture.get(cv.CAP_PROP_FPS)}')
print(f'Start offset: {capture.get(cv.CAP_PROP_POS_FRAMES)}')
print(f'Total frames: {int(capture.get(cv.CAP_PROP_FRAME_COUNT))}')
print(f'Total seconds: {int(capture.get(cv.CAP_PROP_FRAME_COUNT) / (capture.get(cv.CAP_PROP_FPS)))}')

In [ ]:
# Create background subtractor
backSub = cv.createBackgroundSubtractorKNN()

fgMask = None
masks = []

with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
    while True:
        ret, frame = capture.read()
    
        if frame is None:
            break

        pbar.update(1)
    
        fgMask = backSub.apply(frame)
        
        masks.append(fgMask[fgMask > 0].shape[0])

In [ ]:
df = pd.DataFrame({'signal': masks})

In [ ]:
ax = df.plot(figsize=(14, 4))
ax.grid(axis='both', which='both')
plt.minorticks_on()
plt.ylim([0, 100000])
plt.show()

In [ ]:
origin = lambda x: x
bilateral = lambda x, sigma=1<<5: cv.bilateralFilter(x, sigma, sigma * 2, sigma / 2)
gaussian = lambda x: cv.GaussianBlur(x, (5, 5), 0)
blur = lambda x: cv.blur(frame, (5, 5))
median = lambda x: cv.medianBlur(frame, 5)

filters = {
    'original': {'filter': origin, 'subtractor': cv.createBackgroundSubtractorKNN(), 'signal': []},
    'bilateral': {'filter': bilateral, 'subtractor': cv.createBackgroundSubtractorKNN(), 'signal': []},
    'blur': {'filter': blur, 'subtractor': cv.createBackgroundSubtractorKNN(), 'signal': []},
    'gaussian': {'filter': gaussian, 'subtractor': cv.createBackgroundSubtractorKNN(), 'signal': []},
    'median': {'filter': median, 'subtractor': cv.createBackgroundSubtractorKNN(), 'signal': []},
}

capture = cv.VideoCapture(video_file)
with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
    while True:
        # Read frame
        ret, frame = capture.read()
    
        # EOF
        if frame is None:
            break
        pbar.update(1)
    
        for filter_name in filters:
            filtered_frame = filters[filter_name]['filter'](frame)
            mask = filters[filter_name]['subtractor'].apply(filtered_frame)
            filters[filter_name]['signal'].append(mask[mask > 0].shape[0])

In [ ]:
df = pd.DataFrame({
    'orig': filters['original']['signal'],
    'bilateral': filters['bilateral']['signal'],
    'blur': filters['blur']['signal'],
    'gaussian': filters['gaussian']['signal'],
    'median': filters['median']['signal'],
})

ax = df.plot(figsize=(14, 4))
ax.grid(axis='both', which='both')
plt.minorticks_on()
plt.ylim([0, 100000])
plt.show()

In [ ]:
# Kalman
kalman_arr = []
kalman = cv.KalmanFilter(2, 1)
kalman.measurementMatrix = np.array([[1, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.03

# Add kalman data storage
filters.update({
    'kalman': {'filter': None, 'subtractor': None, 'signal': []},
})
# Reset blur filter data storage
filters['blur']['signal'] = []

capture = cv.VideoCapture(video_file)

with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
    while True:
        # Read frame
        ret, frame = capture.read()
    
        # EOF
        if frame is None:
            break
        pbar.update(1)
    
        filtered_frame = filters['blur']['filter'](frame)
        mask = filters[filter_name]['subtractor'].apply(filtered_frame)
        filters['blur']['signal'].append(mask[mask > 0].shape[0])

        measurement = np.array([np.float32(mask[mask > 0].shape[0])])

        kalman.correct(measurement)
        prediction = kalman.predict()
        filters['kalman']['signal'].append(prediction[0][0])

In [ ]:
df = pd.DataFrame({
    'orig': filters['original']['signal'],
    'blur': filters['blur']['signal'],
    'kalman': filters['kalman']['signal'],
})

ax = df.plot(figsize=(14, 4))
ax.grid(axis='both', which='both')
plt.minorticks_on()
plt.ylim([0, 100000])
plt.show()

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

def process_video(video_file):
    capture = cv.VideoCapture(video_file)
    print(f'Processing video: {video_file}')
    print(f'FPS: {capture.get(cv.CAP_PROP_FPS)}')
    print(f'Total frames: {int(capture.get(cv.CAP_PROP_FRAME_COUNT))}')
    
    backSub = cv.createBackgroundSubtractorKNN()
    
    masks = []
    
    # First pass: Get motion detection signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)
            fgMask = backSub.apply(frame)
            masks.append(fgMask[fgMask > 0].shape[0])
    
    df = pd.DataFrame({'signal': masks})
    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Motion Detection Signal for {video_file}')
    plt.show()

    # Kalman Filter setup
    kalman_arr = []
    kalman = cv.KalmanFilter(2, 1)
    kalman.measurementMatrix = np.array([[1, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
    kalman.processNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.03

    # Filtering section with only blur and Kalman
    filters = {
        'blur': {'filter': lambda x: cv.blur(x, (5, 5)), 'subtractor': backSub, 'signal': []},
        'kalman': {'filter': None, 'subtractor': None, 'signal': []},
    }

    # Reset the capture to start again
    capture.set(cv.CAP_PROP_POS_FRAMES, 0)

    # Second pass: Apply filters and get signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)

            # Apply blur filter
            filtered_frame = filters['blur']['filter'](frame)
            mask = filters['blur']['subtractor'].apply(filtered_frame)
            filters['blur']['signal'].append(mask[mask > 0].shape[0])

            # Kalman filter prediction
#             measurement = np.array([np.float32(mask[mask > 0].shape[0])])
#             kalman.correct(measurement)
#             prediction = kalman.predict()
#             filters['kalman']['signal'].append(prediction[0][0])

    df = pd.DataFrame({
        'blur': filters['blur']['signal'],
#         'kalman': filters['kalman']['signal'],
    })

    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Blur Signals for {video_file}')
    plt.show()

# List of video files
video_files = [
    '/kaggle/input/vids-for-motion-detection/Normal_Videos020_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos071_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos093_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos099_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos114_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos149_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos162_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos178_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos222_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos236_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos243_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos244_x264.mp4',
]

# Loop through all video files
for video_file in video_files:
    process_video(video_file)

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

def calculate_snr(signal):
    """Calculate Signal-to-Noise Ratio (SNR)."""
    signal_mean = np.mean(signal)
    noise_std = np.std(signal)
    if noise_std == 0:  # Avoid division by zero
        return float('inf')  # High SNR if there's no noise
    return 10 * np.log10(signal_mean / noise_std)

def process_video(video_file):
    capture = cv.VideoCapture(video_file)
    print(f'Processing video: {video_file}')
    print(f'FPS: {capture.get(cv.CAP_PROP_FPS)}')
    print(f'Total frames: {int(capture.get(cv.CAP_PROP_FRAME_COUNT))}')
    
    backSub = cv.createBackgroundSubtractorKNN()
    
    masks = []
    
    # First pass: Get motion detection signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)
            fgMask = backSub.apply(frame)
            masks.append(fgMask[fgMask > 0].shape[0])
    
    # Calculate SNR for the raw signal
    raw_snr = calculate_snr(masks)
    print(f'Raw SNR for {video_file}: {raw_snr:.2f} dB')
    
    df = pd.DataFrame({'signal': masks})
    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Motion Detection Signal for {video_file}')
    plt.show()

    # Filtering section
    filters = {
        'blur': {'filter': lambda x: cv.blur(x, (5, 5)), 'subtractor': backSub, 'signal': []},
    }

    # Reset the capture to start again
    capture.set(cv.CAP_PROP_POS_FRAMES, 0)

    # Second pass: Apply filters and get signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)

            # Apply blur filter
            filtered_frame = filters['blur']['filter'](frame)
            mask = filters['blur']['subtractor'].apply(filtered_frame)
            filters['blur']['signal'].append(mask[mask > 0].shape[0])

    # Calculate SNR for the blurred signal
    blurred_signal = filters['blur']['signal']
    blurred_snr = calculate_snr(blurred_signal)
    print(f'Blurred SNR for {video_file}: {blurred_snr:.2f} dB')

    # Create a table to display SNR values
    snr_df = pd.DataFrame({
        'Metric': ['Raw SNR', 'Blurred SNR'],
        'Value (dB)': [raw_snr, blurred_snr]
    })
    
    print("\nSNR Comparison Table:")
    print(snr_df)

    # Plotting the blurred signal
    df = pd.DataFrame({
        'blur': filters['blur']['signal'],
    })

    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Blur Signals for {video_file}')
    plt.show()

# List of video files
video_files = [
    '/kaggle/input/vids-for-motion-detection/Normal_Videos020_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos071_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos093_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos099_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos114_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos149_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos162_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos178_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos222_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos236_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos243_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos244_x264.mp4',    
]

# Loop through all video files
for video_file in video_files:
    process_video(video_file)


In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

def process_video(video_file):
    capture = cv.VideoCapture(video_file)
    print(f'Processing video: {video_file}')
    print(f'FPS: {capture.get(cv.CAP_PROP_FPS)}')
    print(f'Total frames: {int(capture.get(cv.CAP_PROP_FRAME_COUNT))}')
    
    backSub = cv.createBackgroundSubtractorKNN()
    
    masks = []
    
    # First pass: Get motion detection signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)
            fgMask = backSub.apply(frame)
            masks.append(fgMask[fgMask > 0].shape[0])
    
    df = pd.DataFrame({'signal': masks})
    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Motion Detection Signal for {video_file}')
    plt.show()

    # Kalman Filter setup
    kalman_arr = []
    kalman = cv.KalmanFilter(2, 1)
    kalman.measurementMatrix = np.array([[1, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
    kalman.processNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.03

    # Filtering section with only blur and Kalman
    filters = {
        'blur': {'filter': lambda x: cv.blur(x, (5, 5)), 'subtractor': backSub, 'signal': []},
        'kalman': {'filter': None, 'subtractor': None, 'signal': []},
    }

    # Reset the capture to start again
    capture.set(cv.CAP_PROP_POS_FRAMES, 0)

    # Second pass: Apply filters and get signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)

            # Apply blur filter
            filtered_frame = filters['blur']['filter'](frame)
            mask = filters['blur']['subtractor'].apply(filtered_frame)
            filters['blur']['signal'].append(mask[mask > 0].shape[0])

            # Kalman filter prediction
            measurement = np.array([np.float32(mask[mask > 0].shape[0])])
            kalman.correct(measurement)
            prediction = kalman.predict()
            filters['kalman']['signal'].append(prediction[0][0])

    df = pd.DataFrame({
        'blur': filters['blur']['signal'],
        'kalman': filters['kalman']['signal'],
    })

    ax = df.plot(figsize=(14, 4))
    ax.grid(axis='both', which='both')
    plt.minorticks_on()
    plt.ylim([0, 400000])
    plt.title(f'Blur Signals for {video_file}')
    plt.show()

# List of video files
video_files = [
    '/kaggle/input/vids-for-motion-detection/Normal_Videos020_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos071_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos093_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos099_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos114_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos149_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos162_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos178_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos222_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos236_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos243_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos244_x264.mp4',
]

# Loop through all video files
for video_file in video_files:
    process_video(video_file)

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

def process_video(video_file, threshold=1000):
    capture = cv.VideoCapture(video_file)
    print(f'Processing video: {video_file}')
    print(f'FPS: {capture.get(cv.CAP_PROP_FPS)}')
    print(f'Total frames: {int(capture.get(cv.CAP_PROP_FRAME_COUNT))}')
    
    backSub = cv.createBackgroundSubtractorKNN()
    
    masks = []
    
    # First pass: Get motion detection signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)
            fgMask = backSub.apply(frame)
            masks.append(fgMask[fgMask > 0].shape[0])
    
    # Initialize Kalman Filter
    kalman = cv.KalmanFilter(2, 1)
    kalman.measurementMatrix = np.array([[1, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
    kalman.processNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.03

    # Filtering section with blur and Kalman
    filters = {
        'blur': {'filter': lambda x: cv.blur(x, (5, 5)), 'subtractor': backSub, 'signal': []},
        'kalman': {'signal': []},
    }

    # Reset the capture to start again
    capture.set(cv.CAP_PROP_POS_FRAMES, 0)

    # Second pass: Apply filters and get signals
    with tqdm(total=capture.get(cv.CAP_PROP_FRAME_COUNT)) as pbar:
        while True:
            ret, frame = capture.read()
            if not ret:  # Check if frame is valid
                break
            pbar.update(1)

            # Apply blur filter
            filtered_frame = filters['blur']['filter'](frame)
            mask = filters['blur']['subtractor'].apply(filtered_frame)
            filters['blur']['signal'].append(mask[mask > 0].shape[0])

            # Kalman filter prediction
            measurement = np.array([np.float32(mask[mask > 0].shape[0])])
            kalman.correct(measurement)
            prediction = kalman.predict()
            filters['kalman']['signal'].append(prediction[0][0])

    df = pd.DataFrame({
        'blur': filters['blur']['signal'],
        'kalman': filters['kalman']['signal'],
    })

    # Apply thresholding
    df['motion_detected_blur'] = df['blur'] > threshold
    df['motion_detected_kalman'] = df['kalman'] > threshold

    # Calculate True Positives, False Positives, False Negatives
    # For example, let's assume you have a true motion label array `true_labels`
    # true_labels = ... # This should be a boolean array indicating true motion (1) or not (0)
    # TP = np.sum(df['motion_detected_blur'] & true_labels)
    # FP = np.sum(df['motion_detected_blur'] & ~true_labels)
    # FN = np.sum(~df['motion_detected_blur'] & true_labels)

    # Plotting the signals
    ax = df[['blur', 'kalman']].plot(figsize=(14, 4), alpha=0.7)
    ax.fill_between(df.index, 0, df['blur'], where=df['motion_detected_blur'], color='yellow', alpha=0.5, label='Detected Motion (Blur)')
    ax.fill_between(df.index, 0, df['kalman'], where=df['motion_detected_kalman'], color='green', alpha=0.5, label='Detected Motion (Kalman)')
    ax.axhline(y=threshold, color='red', linestyle='--', label='Threshold')
    
    plt.title(f'Blur and Kalman Filter Signals with Motion Detection for {video_file}')
    plt.xlabel('Frame Index')
    plt.ylabel('Signal Amplitude')
    plt.legend()
    plt.grid()
    plt.show()

# List of video files
video_files = [
    '/kaggle/input/vids-for-motion-detection/Normal_Videos020_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos071_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos093_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos099_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos114_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos149_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos162_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos178_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos222_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos236_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos243_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos244_x264.mp4',    
]

# Loop through all video files
for video_file in video_files:
    process_video(video_file)


In [ ]:
import cv2 as cv
import pandas as pd
import numpy as np

def label_video_frames(video_file, threshold=1000):
    capture = cv.VideoCapture(video_file)
    frame_labels = []

    # Background subtractor and Kalman filter initialization
    backSub = cv.createBackgroundSubtractorKNN()
    kalman = cv.KalmanFilter(2, 1)
    kalman.measurementMatrix = np.array([[1, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0], [0, 1]], np.float32)
    kalman.processNoiseCov = np.array([[1, 0], [0, 1]], np.float32) * 0.03

    while True:
        ret, frame = capture.read()
        if not ret:
            break

        # Apply blur filter
        blurred_frame = cv.blur(frame, (5, 5))
        
        # Apply background subtraction
        fgMask = backSub.apply(blurred_frame)
        motion_pixels = np.sum(fgMask > 0)

        # Kalman filter prediction and correction
        measurement = np.array([[np.float32(motion_pixels)]])
        kalman.correct(measurement)
        prediction = kalman.predict()

        # Label based on motion pixels after Kalman filtering
        label = 1 if prediction[0][0] > threshold else 0
        frame_labels.append((capture.get(cv.CAP_PROP_POS_FRAMES), label))

    capture.release()
    return frame_labels

# Example usage
video_files = [
    '/kaggle/input/vids-for-motion-detection/Normal_Videos020_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos071_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos093_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos099_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos114_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos149_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos162_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos178_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos209_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos222_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos236_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos243_x264.mp4',
    '/kaggle/input/vids-for-motion-detection/Normal_Videos244_x264.mp4',
]

all_labels = {}
for video_file in video_files:
    labels = label_video_frames(video_file)
    all_labels[video_file] = labels

# Convert to DataFrame for further use
label_df = pd.DataFrame([(vid, frame, label) for vid, frames in all_labels.items() for frame, label in frames], 
                        columns=['video_id', 'frame_index', 'label'])

# Save to CSV
label_df.to_csv('motion_detected_labels.csv', index=False)


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load true labels and detected labels
true_labels_df = pd.read_csv('/kaggle/input/ground-truth-labels/ground_truth_labels.csv')  # Replace with your true labels CSV
detected_labels_df = pd.read_csv('/kaggle/working/motion_detected_labels.csv')  # The one generated with your labeling code

# Merge DataFrames on 'video_id' and 'frame_index'
merged_df = pd.merge(true_labels_df, detected_labels_df, on=['video_id', 'frame_index'], suffixes=('_true', '_detected'))

# Initialize a dictionary to store confusion matrices and metrics for each video
results = {}

# Initialize lists to store true and detected labels for overall metrics
all_y_true = []
all_y_pred = []

# Iterate through each unique video
for video_id in merged_df['video_id'].unique():
    video_data = merged_df[merged_df['video_id'] == video_id]
    
    # Extract true and detected labels
    y_true = video_data['label_true'].values
    y_pred = video_data['label_detected'].values
    
    # Store all true and predicted labels for overall metrics
    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate performance metrics
    report = classification_report(y_true, y_pred, output_dict=True)
    
    # Store results
    results[video_id] = {
        'confusion_matrix': cm,
        'accuracy': report['accuracy'],
        'precision': report['1']['precision'],  # Assuming 1 is the positive class
        'recall': report['1']['recall'],
        'f1-score': report['1']['f1-score']
    }

# Convert results to DataFrame for better readability
results_df = pd.DataFrame.from_dict(results, orient='index')

# Calculate overall performance metrics
overall_accuracy = accuracy_score(all_y_true, all_y_pred)
overall_precision = precision_score(all_y_true, all_y_pred, average='binary', pos_label=1)
overall_recall = recall_score(all_y_true, all_y_pred, average='binary', pos_label=1)
overall_f1_score = f1_score(all_y_true, all_y_pred, average='binary', pos_label=1)

# Print overall performance metrics
print("Overall Performance Metrics:")
print(f"Accuracy: {overall_accuracy:.2f}")
print(f"Precision: {overall_precision:.2f}")
print(f"Recall: {overall_recall:.2f}")
print(f"F1 Score: {overall_f1_score:.2f}")

# Plotting Confusion Matrices and Metrics
for video_id, metrics in results.items():
    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(metrics['confusion_matrix'], annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix for {video_id}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(ticks=[0.5, 1.5], labels=['No Motion', 'Motion'])
    plt.yticks(ticks=[0.5, 1.5], labels=['No Motion', 'Motion'])
    plt.show()

    # Plot performance metrics
    plt.figure(figsize=(8, 5))
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1-score']
    scores = [metrics[m] for m in metrics_to_plot]
    
    bars = plt.bar(metrics_to_plot, scores, color=['blue', 'orange', 'green', 'red'])
    plt.ylim(0, 1)
    plt.title(f'Performance Metrics for {video_id}')
    plt.ylabel('Score')
    
    # Adding score annotations inside the bars
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval / 2, f'{yval:.2f}', ha='center', va='center', color='white')
    
    plt.show()
